In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### Wide Residual Net

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [67]:
#Train the model per the settings specified in the original paper
# os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
# !python train.py --config configs/cifar/wrn.yaml \
#     model.wrn.depth 28 \
#     model.wrn.widening_factor 10 \
#     train.batch_size 128 \
#     train.base_lr 0.1 \
#     train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00 \
#     scheduler.epochs 200
pass

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_3_20 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00 \
    scheduler.epochs 400

[2020-06-24 01:16:52] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_3_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
    

In [7]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00/checkpoint_00400.pth \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr .0008 \
    dataset.name CIFAR10 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR

[2020-06-22 23:40:28] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: 

In [6]:
# Resume training with the un-augmented data
# os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
# #!python train.py --config configs/cifar/resnet.yaml \
# !python train.py --config /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/config.yaml \
#     train.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00300.pth \
#     dataset.name CIFAR10 \
#     train.base_lr .001 \
#     train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150 \
#     scheduler.epochs 150

#### Set LEARNING RATE based on ending LR
#    train.resume True \


In [12]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00_resume400_50/checkpoint_00050.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00_resume400_50/test_results_0050_cifar10

[2020-06-23 12:42:38] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 40/40 [00:51<00:00,  1.28s/it]
[2020-06-23 12:43:30] __main__ INFO: Elapsed 51.36
[2020-06-23 12:43:30] __main__ INFO: Loss 0.2627 Accuracy 0.9403


In [113]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
# !python evaluate.py --config configs/cifar/wrn.yaml \
#     model.wrn.depth 28 \
#     model.wrn.widening_factor 10 \
#     train.batch_size 128 \
#     train.base_lr 0.00001 \
#     test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00_resume300_150/checkpoint_00150.pth \
#     test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00_resume300_150/test_results_0150_cifar10

[2020-06-20 16:32:43] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150/checkpoint_00150.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 26.53it/s]
[2020-06-20 16:32:46] __main__ INFO: Elapsed 2.98
[2020-06-20 16:32:46] __main__ INFO: Loss 0.4499 Accuracy 0.8795


In [14]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20//exp00_resume400_50/checkpoint_00050.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00_resume400_50/test_results_0050_cifar101

[2020-06-23 12:47:22] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5//exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|█████████████████████████████████████████████| 8/8 [00:10<00:00,  1.34s/it]
[2020-06-23 12:47:33] __main__ INFO: Elapsed 10.69
[2020-06-23 12:47:33] __main__ INFO: Loss 0.6479 Accuracy 0.8520


In [114]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
# !python evaluate.py --config configs/cifar/rwrn.yaml \
#     model.wrn.depth 28 \
#     model.wrn.widening_factor 10 \
#     dataset.name CIFAR101 \
#     test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00_resume300_150/checkpoint_00150.pth \
#     test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00_resume300_150/test_results_0150_cifar101

[2020-06-20 16:33:19] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150/checkpoint_00150.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.16it/s]
[2020-06-20 16:33:20] __main__ INFO: Elapsed 0.93
[2020-06-20 16:33:20] __main__ INFO: Loss 0.8206 Accuracy 0.7710


In [16]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00/checkpoint_00400.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00/test_results_0400_cifar101

[2020-06-23 13:04:05] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|█████████████████████████████████████████████| 8/8 [00:10<00:00,  1.33s/it]
[2020-06-23 13:04:16] __main__ INFO: Elapsed 10.63
[2020-06-23 13:04:16] __main__ INFO: Loss 0.8676 Accuracy 0.8145


In [17]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00/checkpoint_00300.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00/test_results_0300

[2020-06-23 13:07:02] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00/checkpoint_00300.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 40/40 [00:51<00:00,  1.28s/it]
[2020-06-23 13:07:54] __main__ INFO: Elapsed 51.38
[2020-06-23 13:07:54] __main__ INFO: Loss 0.3800 Accuracy 0.9128


In [18]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00/checkpoint_00400.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00/test_results_0400_cifar

[2020-06-23 13:09:51] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 40/40 [00:51<00:00,  1.29s/it]
[2020-06-23 13:10:44] __main__ INFO: Elapsed 51.42
[2020-06-23 13:10:44] __main__ INFO: Loss 0.3942 Accuracy 0.9119


In [19]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00/checkpoint_00300.pth \
    dataset.name CIFAR101 \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20/exp00/test_results_0300_cifar101

[2020-06-23 13:13:45] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00/checkpoint_00300.pth
CIFAR 10.1
100%|█████████████████████████████████████████████| 8/8 [00:10<00:00,  1.34s/it]
[2020-06-23 13:13:56] __main__ INFO: Elapsed 10.69
[2020-06-23 13:13:56] __main__ INFO: Loss 0.8150 Accuracy 0.8070


In [20]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['wrn_28_10', 'wrn_28_10', 'wrn_28_10'],
           'Testset': ['cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 200],
           'Loss': [0.2299, 0.1760, 0.3896],
           'Accuracy': [0.9311, 0.9578, 0.8975],
           'Original_Accuracy': [95.9, 95.9, 89.7],
           'Original_CI': [(95.5, 96.3), (95.5, 96.3), (88.3, 91.0)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])

df.to_csv('/home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,wrn_28_10,cifar10,100,0.2299,0.9311,95.9,"(95.5, 96.3)"
1,wrn_28_10,cifar10,200,0.1760,0.9578,95.9,"(95.5, 96.3)"
2,wrn_28_10,cifar10.1,200,0.3896,0.8975,89.7,"(88.3, 91.0)"


In [21]:
import pandas as pd
a = pd.Series(['wrn_28_10_ra_3_20', 400, 'cifar10',])
b = pd.Series(['wrn_28_10_ra_3_20', 300, 'cifar10', ])
c = pd.Series(['wrn_28_10_ra_3_20', 400, 'cifar10.1', ])
d = pd.Series(['wrn_28_10_ra_3_20', 300, 'cifar10.1', ])
    
e = pd.Series(['wrn_28_10_ra_3_20_refined400', 50, 'cifar10.1', ])
f = pd.Series(['wrn_28_10_ra_3_20_refined400', 50, 'cifar10', ])
#g = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10', 0.4499, 0.8795])
#h = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10.1', 0.8206, 0.7710])
               
df_results = pd.concat([a,b,c,d,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 95.9 if row[2] == 'cifar10' else 89.7), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (95.5, 96.3) if row[2] == 'cifar10' else (88.3, 91.0)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,wrn_28_10_ra_2_5,400,cifar10,0.3942,0.9119,95.9,"(95.5, 96.3)"
1,wrn_28_10_ra_2_5,300,cifar10,0.38,0.9128,95.9,"(95.5, 96.3)"
2,wrn_28_10_ra_2_5,400,cifar10.1,0.8676,0.8145,89.7,"(88.3, 91.0)"
3,wrn_28_10_ra_2_5,300,cifar10.1,0.815,0.807,89.7,"(88.3, 91.0)"
4,wrn_28_10_ra_2_5_refined400,50,cifar10.1,0.6479,0.852,89.7,"(88.3, 91.0)"
5,wrn_28_10_ra_2_5_refined400,50,cifar10,0.2627,0.9403,95.9,"(95.5, 96.3)"


In [8]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00/test_results_0400/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[ -7.153804  ,  -0.1832159 ,  -0.69570637, ...,  -0.50926757,
         -5.526208  , -12.987257  ],
       [  2.862379  ,   7.963458  ,  -6.603018  , ...,  -4.740323  ,
         25.90399   ,  -0.52988565],
       [  4.25749   ,   8.408992  ,  -4.3299227 , ...,  -2.3715498 ,
         13.468082  ,   4.5792727 ],
       ...,
       [ -4.7270765 ,  -1.2400844 ,   1.3852903 , ...,  -0.51062894,
         -3.399443  ,  -2.4969094 ],
       [ -2.7640457 ,  14.635863  ,   6.7449965 , ...,  -1.3011913 ,
         -3.036379  ,  -7.061736  ],
       [ -2.6933427 ,   1.8961854 ,  -3.6396854 , ...,  18.63456   ,
         -2.7524152 ,  -3.2204888 ]], dtype=float32)

In [22]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/wrn_28_10_ra_3_20'
path = '/home/ec2-user/SageMaker/experiments/wrn_28_10_ra_3_20'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)